In [1]:
import numpy as np
import binascii

In [2]:
def decimalToBinary(num , places):
    return ('{0:0' +  str(places) + 'b}').format(num)


def binaryToDecimal(binary):
    return int(binary, 2)
        
def xorBinary(binary,key = "00000000"):
    i=0
    encBin = ""
    while(i<8):
        if(binary[i] == '0' and key[i] == '0') or (binary[i] == '1' and key[i] == '1'):
            encBin+="0"
        else:
            encBin+="1"
        i+=1
        
#     print(binary , key , encBin)    
    return encBin
    
def compressBinary(binary):
    i=0
    compressBin = ""
    while(i<8):
        if(i%2 == 0):
            if((binary[i] == '0' and binary[i+1] == '0') or (binary[i] == '1' and binary[i+1] == '1')):
                compressBin+="0"
            else:
                compressBin+="1"
        i+=1
        
    return compressBin

def to_twoscomplement(bits, value):
    if value < 0:
        value = ( 1<<bits ) + value
    formatstring = '{:0%ib}' % bits
    return formatstring.format(value)


# Code for Importing Data

In [3]:
import pandas as pd
article_read = pd.read_csv('dataset.csv',delimiter = ',')

In [4]:
# new_dataset = pd.Series(list(article_read['Temp.'].sample(101 , random_state=10)))
new_dataset = article_read['Temp.']

# new_dataset

# Code For Delta Coding

In [5]:
new_dataset.describe()

count    928991.000000
mean         27.283373
std           0.904537
min          24.434400
25%          26.764900
50%          27.328400
75%          27.868700
max          30.325400
Name: Temp., dtype: float64

In [6]:
int_dataset = new_dataset * 10000

In [7]:
int_dataset = int_dataset.astype('int64')

In [8]:
int_dataset.head()

0    262257
1    262308
2    262365
3    262416
4    262462
Name: Temp., dtype: int64

In [9]:
diff_dataset = [int_dataset[0]]
for i in range(1 , len(int_dataset)) :
        diff_dataset.append(int_dataset[i] - int_dataset[i-1])

In [10]:
diff_dataset = pd.Series(diff_dataset)

In [11]:
diff_dataset.head()

0    262257
1        51
2        57
3        51
4        46
dtype: int64

In [12]:
import seaborn as sns
%matplotlib inline


In [13]:
diff_dataset.describe()

count    928991.000000
mean          0.298587
std         295.352490
min      -41416.000000
25%          -7.000000
50%           0.000000
75%           6.000000
max      262257.000000
dtype: float64

In [14]:
len(diff_dataset)

928991

In [15]:
((diff_dataset > 64) | (diff_dataset < -64)).sum() 

24877

In [16]:
24877/928991 * 100

2.6778515615328886

In [17]:
(2.6778515615328886 * 24 + (100 - 2.6778515615328886) *  8) / (100 * 20)


0.4214228124922631

In [21]:
bytesToSend = []
for i in diff_dataset :
    if(i > 64 or i < -64) :
        # Send 3 Bytes
        number = decimalToBinary(abs(i) , 21)
        if(i < 0):
            firstByte = '01' + number[1:7]
        else :
            firstByte = '00' + number[1:7]
        secondByte = '0' + number[7:14]
        thirdByte = '0' + number[14:21]
        bytesToSend.append(firstByte)
        bytesToSend.append(secondByte)
        bytesToSend.append(thirdByte)
    else :
        number = decimalToBinary(abs(i) , 6)
        if(i < 0):
            firstByte = '11' + number
        else :
            firstByte = '10' + number
        bytesToSend.append(firstByte)

            
        
        
        
        
        

In [22]:
len(bytesToSend)

978745

In [23]:
i = 0
current = 0
currentByte = ''

dataSet = []
last = 0

while(i < len(bytesToSend)) :
    recvByte = bytesToSend[i]
    if recvByte[0] == '0' :
        current += 1
        currentByte += recvByte[1:]
        if current == 3 :
            dataPoint = int(currentByte[1:], 2)
            if(currentByte[0]) == '1' :
                dataPoint *= -1
            
            dataPoint += last
            dataSet.append(dataPoint)
            last = dataPoint
            current = 0
            currentByte = ''
    else :
        dataPoint = int(recvByte[2:], 2)
        if(recvByte[1]) == '1' :
            dataPoint *= -1
        if i > 15 and i < 21 :
            print(i , dataPoint , recvByte) 
        dataPoint += last
        dataSet.append(dataPoint)
        last = dataPoint
    
    i+=1
    
dataSet = pd.Series(dataSet)

16 12 10001100
17 10 10001010
18 3 10000011
19 -1 11000001
20 0 10000000


In [24]:
pd.concat([dataSet, int_dataset , diff_dataset], axis=1)

,0,Temp.,1
0,262257,262257,262257
1,262308,262308,51
2,262365,262365,57
3,262416,262416,51
4,262462,262462,46
5,262503,262503,41
6,262541,262541,38
7,262575,262575,34
8,262606,262606,31
9,262626,262626,20


In [31]:
(dataSet != int_dataset).sum() # Zero Denotes No Mis Match.

0

In [26]:
# Original Data Sent => Considering 20Bits Per Data Point
originalDataSize = len(int_dataset) * 20 / 8

In [27]:
# New Data Size 
compressedDataSize = len(bytesToSend)

In [28]:
ratio = compressedDataSize / originalDataSize

In [29]:
ratio

0.4214228124922631

In [32]:
(1 - ratio) * 100 # Reduction Percatage

57.85771875077369